In [1]:
#CNN is a form of deep learning that specializes in pattern recognition 

#ask Praveen to explain pooling for you 

'/Users/daniellejenkins17/Documents/GitHub/Pneumonia-Classification'

In [70]:
pwd

'/Users/daniellejenkins17/Documents/GitHub/Pneumonia-Classification'

In [71]:
import numpy as np             
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt

In [72]:
import glob
import random
from random import randint
from pathlib import Path

In [73]:
import numpy as np # forlinear algebra
import matplotlib.pyplot as plt #for plotting things
import os
from PIL import Image

In [74]:
# Keras Libraries
import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.preprocessing import image_dataset_from_directory
from sklearn.metrics import classification_report, confusion_matrix

In [75]:
from tensorflow.keras import datasets
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense # creates densely connected layer object
from tensorflow.keras.layers import Flatten # takes 2D input and turns into 1D array

from tensorflow.keras.layers import Conv2D # convolution layer
from tensorflow.keras.layers import MaxPooling2D # max pooling layer

In [76]:
from sklearn.model_selection import train_test_split
from keras.callbacks import ReduceLROnPlateau

In [77]:
train_data_dir = "data//train"
test_data_dir = "data//test"
val_data_dir = "data//val"

In [78]:
trn_data = image_dataset_from_directory(train_data_dir)
tst_data = image_dataset_from_directory(test_data_dir)
val_data = image_dataset_from_directory(val_data_dir)

#the classes are pneumonia or no pneumonia 

Found 5216 files belonging to 2 classes.
Found 624 files belonging to 2 classes.
Found 16 files belonging to 2 classes.


In [82]:
(trn_data, train_data_dir), (tst_data, test_data_dir) = datasets.mnist.load_data()

11493376/11490434 [==============================] - 1s 0us/step


In [83]:
trn_data.shape

(60000, 28, 28)

In [84]:
tst_data.shape

(10000, 28, 28)

In [85]:
train_data_dir = to_categorical(train_data_dir)

In [ ]:
#Conv2D takes tensors of shape:
#(image_height, image_width, color_channels) for each image
#MNIST has one color channel (grayscale).

In [86]:
trn_data = trn_data.reshape((60000, 28, 28, 1))
tst_data = tst_data.reshape((10000, 28, 28, 1))

In [87]:
trn_data.shape

(60000, 28, 28, 1)

In [ ]:
#next step is normalizing the data 

#scale at 255 

#figure out what convolution vs pooling is 

In [88]:
# Normalize pixel values to be between 0 and 1
trn_data, tst_data = trn_data / 255, tst_data / 255

In [89]:
model = Sequential()
# define 3x3 filter window sizes. Create 32 filters.
model.add(Conv2D(filters=32,
                        kernel_size=(3, 3),
                        activation='relu',
                        input_shape=(28, 28, 1)))
# max pool in 2x2 window
model.add(MaxPooling2D(pool_size=(2, 2)))
# define 3x3 filter window sizes. Create 64 filters.
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))

# transition to dense fully-connected part of network
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [90]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

In [92]:
#compiling and training the model

model.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])

In [93]:
xrays = model.fit(trn_data, train_data_dir, epochs= 20, validation_split = 0.2, batch_size=32)

Epoch 1/20
1500/1500 [==============================] - 22s 15ms/step - loss: 0.1676 - accuracy: 0.9486 - val_loss: 0.0718 - val_accuracy: 0.9787
Epoch 2/20
1500/1500 [==============================] - 21s 14ms/step - loss: 0.0529 - accuracy: 0.9831 - val_loss: 0.0550 - val_accuracy: 0.9836
Epoch 3/20
1500/1500 [==============================] - 21s 14ms/step - loss: 0.0359 - accuracy: 0.9887 - val_loss: 0.0436 - val_accuracy: 0.9876
Epoch 4/20
1500/1500 [==============================] - 22s 15ms/step - loss: 0.0291 - accuracy: 0.9906 - val_loss: 0.0417 - val_accuracy: 0.9890
Epoch 5/20
1500/1500 [==============================] - 22s 15ms/step - loss: 0.0222 - accuracy: 0.9927 - val_loss: 0.0427 - val_accuracy: 0.9877
Epoch 6/20
1500/1500 [==============================] - 23s 16ms/step - loss: 0.0182 - accuracy: 0.9941 - val_loss: 0.0450 - val_accuracy: 0.9885
Epoch 7/20
1500/1500 [==============================] - 25s 17ms/step - loss: 0.0138 - accuracy: 0.9951 - val_loss: 0.0429 -

In [95]:
#evaluating the model

_, test_acc = model.evaluate(tst_data, to_categorical(test_data_dir))

313/313 [==============================] - 6s 20ms/step - loss: 0.0428 - accuracy: 0.9931


In [96]:
#improving the model with regularization

from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dropout

# a new layer that rescales/normalizes the activations after each layer.
from tensorflow.keras.layers import BatchNormalization

In [97]:
model2 = Sequential()
# define 3x3 filter window sizes. Create 32 filters.
model2.add(Conv2D(filters=32,
                        kernel_size=(3, 3),
                        activation='relu',
                        input_shape=(28, 28, 1), kernel_regularizer = l2(1e-2) ))

model2.add(BatchNormalization())
# max pool in 2x2 window
model2.add(MaxPooling2D(pool_size=(2, 2)))
# define 3x3 filter window sizes. Create 64 filters.
model2.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer = l2(1e-2) ))

model2.add(BatchNormalization())
model2.add(MaxPooling2D((2, 2)))
model2.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer = l2(1e-2)))

model2.add(BatchNormalization())

# transition to dense fully-connected part of network
model2.add(Flatten())
model2.add(Dense(64, activation='relu', kernel_regularizer = l2(1e-2)))
model2.add(BatchNormalization())
model2.add(Dense(10, activation='softmax'))

model2.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])

In [98]:
history_reg = model2.fit(trn_data, train_data_dir, epochs= 20, validation_split = 0.2, batch_size=64)

Epoch 1/20
750/750 [==============================] - 110s 147ms/step - loss: 0.8306 - accuracy: 0.9606 - val_loss: 0.2606 - val_accuracy: 0.9735
Epoch 2/20
750/750 [==============================] - 1173s 2s/step - loss: 0.2125 - accuracy: 0.9762 - val_loss: 0.2461 - val_accuracy: 0.9643
Epoch 3/20
750/750 [==============================] - 30s 41ms/step - loss: 0.1940 - accuracy: 0.9765 - val_loss: 0.2943 - val_accuracy: 0.9450
Epoch 4/20
750/750 [==============================] - 31s 42ms/step - loss: 0.1834 - accuracy: 0.9787 - val_loss: 0.2471 - val_accuracy: 0.9572
Epoch 5/20
750/750 [==============================] - 29s 39ms/step - loss: 0.1773 - accuracy: 0.9790 - val_loss: 0.1976 - val_accuracy: 0.9732
Epoch 6/20
750/750 [==============================] - 29s 39ms/step - loss: 0.1673 - accuracy: 0.9799 - val_loss: 0.1712 - val_accuracy: 0.9781
Epoch 7/20
750/750 [==============================] - 29s 39ms/step - loss: 0.1574 - accuracy: 0.9815 - val_loss: 0.1693 - val_accurac